# Groundwater level prediction

In [ ]:
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit, GridSearchCV
import matplotlib.pyplot as plt
from operator import attrgetter

# allow plots to appear within the notebook
%matplotlib inline

## The Data

Groundwater level sensors (Ljubljana polje aquifer) with corresponding weather data for Ljubljana from 2010-01-01 to 2017-12-31:
- `85065` // Lj. - Flajšmanova (Fip-1/04)
- `85012` // Roje (V-01)
- `85064` // Lj-Bratislavska (Brp-1a/04)
- `85030` // Kleče (0541)

Data columns:
- `date`: measurement date
- `level`: groundwater level `[m]`
- `sun_duration`: sun duration `[h]`
- `cloud_cover`: cloud cover `[%]`
- `precipitation`: precipitation `[mm]`
- `snow_accumulation`: daily snow accumulation `[cm]`
- `snow_depth`: snow blanket depth `[cm]`
- `temperature_avg`: average temperature `[°C]`
- `temperature_min`: minimum temperature `[°C]`
- `temperature_max`: maximum temperature `[°C]`

In [ ]:
# Load the data
sensor_id = 85065
df = pd.read_csv(f'../data/ground/{sensor_id}.csv', index_col='date')

## Water level change

We are modelling water level change, not absolute water level. 

In [ ]:
diff = df['level'] - df['level'].shift(1)
df.insert(1, 'level_diff', diff)
#odsrani prvo vrstico ker nima razlike
df = df[1:]
df

## Feature construction

In [ ]:
#generira dneve ker 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20... ker bolj ko gremo nazaj manj gosto nas zanimajo dnevi
def get_range(min, max):
    r = range(min, max)
    r = []
    i = min
    while i <= max:
        r.append(i)
        #navzdol zaokroži logaritem
        e = math.floor(math.log10(i))
        d = 10 ** e
        if i < 10 ** (e + 1) / 2:
            d = math.ceil(d / 2)
        i += d
    return r


def shift_features(dataset, blacklist, max_shift=20, horizon=3):
    days = get_range(1, max_shift)
    #print(days)
    for feature_name in list(dataset.columns):
        #print(feature_name)
        for i in days:
            #generira stolpec ki pove kakšen je bil parameter i dni nazaj (če feature ni v blacklistu)
            #če je vremenska napoved možna za 3 dni ne bomo mogli za 3 dni naprej dobiti podatkov o levelu podtalnice
            #izpred 1 ali dva dni, lahko pa bomo dobili podatek izpred treh dni    
            if feature_name in blacklist and i < horizon:
                continue
            #dodamo stolpec nekega feature name, ki je zamaknjen za i dni
            dataset[f'{feature_name}_shift_{str(i)}d'] = dataset[feature_name].shift(i)


def average_features(dataset, blacklist, max_average=20):
    days = get_range(2, max_average)
    #print(days)
    for feature_name in list(dataset.columns):
        for i in days:
            if feature_name in blacklist:
                continue
            #za nek feuture name gremo z oknom velikosti i (za i zadnjih dni) čez podatke, in po njih izračunamo povprečje
            dataset[f'{feature_name}_average_{str(i)}d'] = dataset[feature_name].rolling(i).sum() / i

In [ ]:
prediction_horizon = 3
max_shift = max_average = 20

shift_features(df, ['level', 'level_diff'], max_shift, prediction_horizon)
average_features(df, ['level', 'level_diff'], max_average)

# Drop all rows containing NaNs generated during feature construction.
min_row = max_shift + max_average - 1
#iloc omogoča iskanje kot v tabeli
df = df.iloc[min_row:, :]
display(df)

'''#dodamo ciljne spremenljivke za 5 prediction horiznov
df.insert(1, 'h1', df['level_diff'].shift(-1))
df.insert(1, 'h2', df['level_diff'].shift(-2))
df.insert(1, 'h3', df['level_diff'].shift(-3))
df.insert(1, 'h5', df['level_diff'].shift(-5))
df=df.iloc[:-5, :]

display(df)'''





## Feature selection

In [ ]:

# Preselected features
selected_features = [
    'level_diff_shift_3d',
    'level_diff_shift_4d',
    'level_diff_shift_5d',
    'precipitation_average_4d',
    'precipitation_average_3d',
    'precipitation_average_5d',
    'precipitation_average_6d',
    'precipitation_average_7d',
    'precipitation_shift_1d_average_2d',
    'precipitation_shift_1d_average_3d',
    'precipitation_average_2d',
    'precipitation_shift_1d_average_4d',
    'precipitation_average_8d',
    'precipitation_shift_1d_average_5d',
    'precipitation_average_9d',
    'precipitation_shift_1d',
    'precipitation_average_10d',
    'precipitation_shift_1d_average_6d',
    'precipitation_shift_1d_average_7d',
    'precipitation_shift_1d_average_8d'
]

'''# Additional k-best features
k = 15
#izberemo vse featurje
df_X = df.iloc[:, 6:]
#pretvori dataset v tabelo in jo casta v float
X = df_X.values.astype(float)
y = df['level_diff'].values.astype(float)
y_h1 = df['h1'].values.astype(float)
y_h2 = df['h2'].values.astype(float)
y_h3 = df['h3'].values.astype(float)
y_h5 = df['h5'].values.astype(float)

#za vsak prediction horizon izberemo ustrezne featurje
X_h1=df_X.values.astype(float)
selected_features_h1 = [
    f'level_shift_{prediction_horizon}d',
    f'level_shift_{prediction_horizon + 1}d',
    f'level_shift_{prediction_horizon + 2}d',
    f'level_diff_shift_{prediction_horizon}d',
    f'level_diff_shift_{prediction_horizon + 1}d',
    f'level_diff_shift_{prediction_horizon + 2}d',
    'sun_duration',
    'cloud_cover',
    'precipitation',
    'snow_accumulation',
    'snow_depth',
    'temperature_avg',
    'temperature_min',
    'temperature_max']
fs_model_h1 = SelectKBest(f_regression, k=k).fit(X_h1, y_h1)
#argsort returns indices that would sort the array
#[::-1] reverses the array
feature_indices_h1 = fs_model_h1.scores_.argsort()[::-1][0:k]
#če featurjev še ni na seznamu jih dodamo
for feature in df_X.columns[feature_indices_h1]:
    if feature not in selected_features_h1:
        selected_features_h1.append(feature)
print(1)
print(selected_features_h1)

X_h2=df_X.values.astype(float)
selected_features_h2 = [
    f'level_shift_{prediction_horizon}d',
    f'level_shift_{prediction_horizon + 1}d',
    f'level_shift_{prediction_horizon + 2}d',
    f'level_diff_shift_{prediction_horizon}d',
    f'level_diff_shift_{prediction_horizon + 1}d',
    f'level_diff_shift_{prediction_horizon + 2}d',
    'sun_duration',
    'cloud_cover',
    'precipitation',
    'snow_accumulation',
    'snow_depth',
    'temperature_avg',
    'temperature_min',
    'temperature_max'
]
fs_model_h2 = SelectKBest(f_regression, k=k).fit(X_h2, y_h2)
feature_indices_h2 = fs_model_h2.scores_.argsort()[::-1][0:k]
for feature in df_X.columns[feature_indices_h2]:
    if feature not in selected_features_h2:
        selected_features_h2.append(feature)
print(2)
print(selected_features_h2)

X_h3=df_X.values.astype(float)
selected_features_h3 = [
    f'level_shift_{prediction_horizon}d',
    f'level_shift_{prediction_horizon + 1}d',
    f'level_shift_{prediction_horizon + 2}d',
    f'level_diff_shift_{prediction_horizon}d',
    f'level_diff_shift_{prediction_horizon + 1}d',
    f'level_diff_shift_{prediction_horizon + 2}d',
    'sun_duration',
    'cloud_cover',
    'precipitation',
    'snow_accumulation',
    'snow_depth',
    'temperature_avg',
    'temperature_min',
    'temperature_max'
]
fs_model_h3 = SelectKBest(f_regression, k=k).fit(X_h3, y_h3)
feature_indices_h3 = fs_model_h3.scores_.argsort()[::-1][0:k]
for feature in df_X.columns[feature_indices_h3]:
    if feature not in selected_features_h3:
        selected_features_h3.append(feature)
print(3)
print(selected_features_h3)

X_h5=df_X.values.astype(float)
selected_features_h5 = [
    f'level_shift_{prediction_horizon}d',
    f'level_shift_{prediction_horizon + 1}d',
    f'level_shift_{prediction_horizon + 2}d',
    f'level_diff_shift_{prediction_horizon}d',
    f'level_diff_shift_{prediction_horizon + 1}d',
    f'level_diff_shift_{prediction_horizon + 2}d',
    'sun_duration',
    'cloud_cover',
    'precipitation',
    'snow_accumulation',
    'snow_depth',
    'temperature_avg',
    'temperature_min',
    'temperature_max'
]
fs_model_h5 = SelectKBest(f_regression, k=k).fit(X_h5, y_h5)
feature_indices_h5 = fs_model_h5.scores_.argsort()[::-1][0:k]
for feature in df_X.columns[feature_indices_h5]:
    if feature not in selected_features_h5:
        selected_features_h5.append(feature)
print(5)
print(selected_features_h5)'''

## fastener feature selection

In [ ]:
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from typing import Dict, List, Callable, Any, Tuple, Optional, \
    Counter as CounterType, Set
from src.random_utils import shuffle
from src import random_utils
from src.item import Item, EvalItem, Result, Population, flatten_population, FitnessFunction, \
    Genes, EvalItem, RandomFlipMutationStrategy, RandomEveryoneWithEveryone, \
    IntersectionMating, UnionMating, IntersectionMatingWithInformationGain, \
    IntersectionMatingWithWeightedRandomInformationGain, UnevaluatedPopulation, \
    MatingStrategy, MutationStrategy, MatingSelectionStrategy
from src import fastener



general_model = LinearRegression

#sets number of samples and number of semples used for testing
n_sample=df.shape[0]
n_test=int(n_sample*0.8)

le = preprocessing.LabelEncoder()
le.fit(df['h1'].values.astype(float))

'''labels_train=le.transform(df['h1'].values.astype(float)[:n_test])
labels_test=le.transform(df['h1'].values.astype(float)[n_test:])
'''
labels_train=df['h1'].values.astype(float)[:n_test]
labels_test=df['h1'].values.astype(float)[n_test:]

XX_train=df.to_numpy()[:n_test, 6:]
XX_test=df.to_numpy()[n_test:, 6:]


def eval_fun(model: Any, genes: "Genes", shuffle_indices: Optional[List[int]] = None) -> "Result":
    test_data = XX_test[:, genes]
    if shuffle_indices:
        test_data = test_data.copy()
        for j in shuffle_indices:
            shuffle(test_data[:, j])
    pred = model.predict(test_data)
    res = Result(r2_score(labels_test, pred))
    return res

number_of_genes = XX_train.shape[1]

initial_genes = [
    [0]
]
# Select mating strategies
mating = RandomEveryoneWithEveryone(pool_size=5, mating_strategy=IntersectionMatingWithWeightedRandomInformationGain(regression=True))

# Random mutation
mutation = RandomFlipMutationStrategy(1 / number_of_genes)

entropy_optimizer = fastener.EntropyOptimizer(
    general_model, XX_train, labels_train, eval_fun,
    number_of_genes, mating, mutation, initial_genes=initial_genes,
    config=fastener.Config(output_folder="h1_tree_c", random_seed=2020, reset_to_pareto_rounds=5)
)
#entropy_optimizer.config.number_of_rounds=10
entropy_optimizer.mainloop()

In [ ]:
from src.random_utils import shuffle
from src import random_utils
from src.item import Item, EvalItem, Result, Population, flatten_population, FitnessFunction, \
    Genes, EvalItem, RandomFlipMutationStrategy, RandomEveryoneWithEveryone, \
    IntersectionMating, UnionMating, IntersectionMatingWithInformationGain, \
    IntersectionMatingWithWeightedRandomInformationGain, UnevaluatedPopulation, \
    MatingStrategy, MutationStrategy, MatingSelectionStrategy
from src import fastener

object = pd.read_pickle(r'log/h1_linear_regression/generation_1000.pickle')
values=object.front.values()
print(values)
m=0
for v in values:
    if v.result.score>m:
        m=v.result.score
        win=v
feat=df.iloc[:, 6:]
selected_features_h1 =feat.iloc[:, win.genes].columns

In [ ]:
object = pd.read_pickle(r'log/h1_tree_c/generation_1000.pickle')
values=object.front.values()
m=0
for v in values:
    if v.result.score>m:
        m=v.result.score
        win=v
feat=df.iloc[:, 6:]
selected_features_h1 =feat.iloc[:, win.genes].columns

# Modelling

## Random forest

In [ ]:
display(df)

In [ ]:
n_folds=10
#df_X = df.iloc[:, 6:]
#crete target values for five different horizons
#select columns with features
X_h1 = df_X[selected_features].values.astype(float)
y_h1 = df['level_diff'].values.astype(float)
'''X_h2 = df_X[selected_features].values.astype(float)
X_h3 = df_X[selected_features].values.astype(float)
X_h5 = df_X[selected_features].values.astype(float)'''



def r2_calculation_random_forest():
    #splits data considering it is a time series
    timeSeriesCV=TimeSeriesSplit(n_splits=n_folds)
    
    #, max_depth = 10, max_features = 'auto', min_samples_leaf = 4, min_samples_split = 10
    model=RandomForestRegressor(n_estimators=100)
    cvs=cross_val_score(model, X_h1, y_h1, scoring='r2', cv=timeSeriesCV)
    print("h1:")
    print("R2: ", cvs.mean(), " stdev: ", cvs.std())

    '''model=RandomForestRegressor(n_estimators=100)
    cvs=cross_val_score(model, X_h2, y_h2, scoring='r2', cv=timeSeriesCV)
    print("h2:")
    print("R2: ", cvs.mean(), " stdev: ", cvs.std())

    model=RandomForestRegressor(n_estimators=100)
    cvs=cross_val_score(model, X_h3, y_h3, scoring='r2', cv=timeSeriesCV)
    print("h3:")
    print("R2: ", cvs.mean(), " stdev: ", cvs.std())

    model=RandomForestRegressor(bootstrap=False, n_estimators=100)
    cvs=cross_val_score(model, X_h5, y_h5, scoring='r2', cv=timeSeriesCV)
    print("h5:")
    print("R2: ", cvs.mean(), " stdev: ", cvs.std())'''
    
def grid_search_random_forest():
    #splits data considering it is a time series
    timeSeriesCV=TimeSeriesSplit(n_splits=n_folds)
    param_search={'max_depth': [10, 50, 70, 100, None], 'max_features': ['auto', 'sqrt'], 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],}
    model=RandomForestRegressor(n_estimators=100)
    gsearch = GridSearchCV(estimator=model, cv=timeSeriesCV, param_grid=param_search, verbose=20, scoring='r2', n_jobs=-1)
    gsearch.fit(X, y_h1)

    #print results
    print(gsearch.best_score_)
    print(gsearch.best_params_)

### displaying results

In [ ]:
def display_results_random_forest():    
    n_samples=df_X.shape[0]
    n_train=int(n_samples*0.9)

    #horizon 1
    X_h1_train = df_X[selected_features_h1].values.astype(float)[:n_train, :]
    X_h1_test = df_X[selected_features_h1].values.astype(float)[n_train:, :]
    y_h1_train = df['h1'].values.astype(float)[:n_train]
    y_h1_test = df['h1'].values.astype(float)[n_train:]

    model=RandomForestRegressor(n_estimators=100)
    model.fit(X_h1_train, y_h1_train)
    res_h1=model.predict(X_h1_test)

    predicted=plt.plot(range(n_samples-n_train), res_h1, label='predicted')
    real=plt.plot(range(n_samples-n_train), y_h1_test, label='real')
    plt.legend()
    plt.title('HORIZON1: difference')
    plt.show()

    real_level=df['level'].values.astype(float)[n_train:]
    predicted_level=[real_level[0]]
    for i in range(1, n_samples-n_train):
        predicted_level.append(res_h1[i]+predicted_level[-1])

    predicted_level_line=plt.plot(range(n_samples-n_train), predicted_level, label='predicted')
    real_level_line=plt.plot(range(n_samples-n_train), real_level, label='real')
    plt.legend()
    plt.title('HORIZON1: level')
    plt.show()

    #horizon 2
    X_h2_train = df_X[selected_features_h2].values.astype(float)[:n_train, :]
    X_h2_test = df_X[selected_features_h2].values.astype(float)[n_train:, :]
    y_h2_train = df['h2'].values.astype(float)[:n_train]
    y_h2_test = df['h2'].values.astype(float)[n_train:]

    model=RandomForestRegressor(n_estimators=100)
    model.fit(X_h2_train, y_h2_train)
    res_h2=model.predict(X_h2_test)

    predicted=plt.plot(range(n_samples-n_train), res_h2, label='predicted')
    real=plt.plot(range(n_samples-n_train), y_h2_test, label='real')
    plt.legend()
    plt.title('HORIZON2: difference')
    plt.show()

    real_level=df['level'].values.astype(float)[n_train:]
    predicted_level=[real_level[0]]
    for i in range(1, n_samples-n_train):
        predicted_level.append(res_h2[i]+predicted_level[-1])

    predicted_level_line=plt.plot(range(n_samples-n_train), predicted_level, label='predicted')
    real_level_line=plt.plot(range(n_samples-n_train), real_level, label='real')
    plt.legend()
    plt.title('HORIZON2: level')
    plt.show()

    #horizon 3
    X_h3_train = df_X[selected_features_h3].values.astype(float)[:n_train, :]
    X_h3_test = df_X[selected_features_h3].values.astype(float)[n_train:, :]
    y_h3_train = df['h3'].values.astype(float)[:n_train]
    y_h3_test = df['h3'].values.astype(float)[n_train:]

    model=RandomForestRegressor(n_estimators=100)
    model.fit(X_h3_train, y_h3_train)
    res_h3=model.predict(X_h3_test)

    predicted=plt.plot(range(n_samples-n_train), res_h3, label='predicted')
    real=plt.plot(range(n_samples-n_train), y_h3_test, label='real')
    plt.legend()
    plt.title('HORIZON3: difference')
    plt.show()

    real_level=df['level'].values.astype(float)[n_train:]
    predicted_level=[real_level[0]]
    for i in range(1, n_samples-n_train):
        predicted_level.append(res_h3[i]+predicted_level[-1])

    predicted_level_line=plt.plot(range(n_samples-n_train), predicted_level, label='predicted')
    real_level_line=plt.plot(range(n_samples-n_train), real_level, label='real')
    plt.legend()
    plt.title('HORIZON3: level')
    plt.show()

    #horizon 5
    X_h5_train = df_X[selected_features_h5].values.astype(float)[:n_train, :]
    X_h5_test = df_X[selected_features_h5].values.astype(float)[n_train:, :]
    y_h5_train = df['h5'].values.astype(float)[:n_train]
    y_h5_test = df['h5'].values.astype(float)[n_train:]

    model=RandomForestRegressor(n_estimators=100)
    model.fit(X_h5_train, y_h5_train)
    res_h5=model.predict(X_h5_test)

    predicted=plt.plot(range(n_samples-n_train), res_h5, label='predicted')
    real=plt.plot(range(n_samples-n_train), y_h5_test, label='real')
    plt.legend()
    plt.title('HORIZON5: difference')
    plt.show()

    real_level=df['level'].values.astype(float)[n_train:]
    predicted_level=[real_level[0]]
    for i in range(1, n_samples-n_train):
        predicted_level.append(res_h5[i]+predicted_level[-1])

    predicted_level_line=plt.plot(range(n_samples-n_train), predicted_level, label='predicted')
    real_level_line=plt.plot(range(n_samples-n_train), real_level, label='real')
    plt.legend()
    plt.title('HORIZON5: level')
    plt.show()

In [ ]:
"""r2 = r2_score(y_test, y_pred)
r2"""

## Linear regression

In [ ]:
n_folds=10

#crete target values for five different horizons
#select columns with features
X_h1 = df_X[selected_features].values.astype(float)
X_h2 = df_X[selected_features].values.astype(float)
X_h3 = df_X[selected_features].values.astype(float)
X_h5 = df_X[selected_features].values.astype(float)


def r2_calculation_linear_regression():
    
    #splits data considering it is a time series
    timeSeriesCV=TimeSeriesSplit(n_splits=n_folds)
    
    model=LinearRegression(normalize=True)
    cvs=cross_val_score(model, X_h1, y_h1, scoring='r2', cv=timeSeriesCV)
    print("h1:")
    print("R2: ", cvs.mean(), " stdev: ", cvs.std())

    model=LinearRegression()
    cvs=cross_val_score(model, X_h2, y_h2, scoring='r2', cv=timeSeriesCV)
    print("h2:")
    print("R2: ", cvs.mean(), " stdev: ", cvs.std())

    model=LinearRegression()
    cvs=cross_val_score(model, X_h3, y_h3, scoring='r2', cv=timeSeriesCV)
    print("h3:")
    print("R2: ", cvs.mean(), " stdev: ", cvs.std())

    model=LinearRegression()
    cvs=cross_val_score(model, X_h5, y_h5, scoring='r2', cv=timeSeriesCV)
    print("h5:")
    print("R2: ", cvs.mean(), " stdev: ", cvs.std())
    
def grid_search_linear_regression():
    param_search={'max_depth': [10, 50, 70, 100, None], 'max_features': ['auto', 'sqrt'], 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],}
    model=LinearRegression()
    gsearch = GridSearchCV(estimator=model, cv=timeSeriesCV, param_grid=param_search, verbose=20, scoring='r2', n_jobs=7)
    gsearch.fit(X, y_h1)

    #print results
    print(gsearch.best_score_)
    print(gsearch.best_params_)

### displaying results(linear regression)

In [ ]:
def display_results_linear_regression():    
    n_samples=df_X.shape[0]
    n_train=int(n_samples*0.9)

    #horizon 1
    X_h1_train = df_X[selected_features_h1].values.astype(float)[:n_train, :]
    X_h1_test = df_X[selected_features_h1].values.astype(float)[n_train:, :]
    y_h1_train = df['h1'].values.astype(float)[:n_train]
    y_h1_test = df['h1'].values.astype(float)[n_train:]

    model=LinearRegression()
    model.fit(X_h1_train, y_h1_train)
    res_h1=model.predict(X_h1_test)

    predicted=plt.plot(range(n_samples-n_train), res_h1, label='predicted')
    real=plt.plot(range(n_samples-n_train), y_h1_test, label='real')
    plt.legend()
    plt.title('HORIZON1: difference')
    plt.show()

    real_level=df['level'].values.astype(float)[n_train:]
    predicted_level=[real_level[0]]
    for i in range(1, n_samples-n_train):
        predicted_level.append(res_h1[i]+predicted_level[-1])

    predicted_level_line=plt.plot(range(n_samples-n_train), predicted_level, label='predicted')
    real_level_line=plt.plot(range(n_samples-n_train), real_level, label='real')
    plt.legend()
    plt.title('HORIZON1: level')
    plt.show()

    #horizon 2
    X_h2_train = df_X[selected_features_h2].values.astype(float)[:n_train, :]
    X_h2_test = df_X[selected_features_h2].values.astype(float)[n_train:, :]
    y_h2_train = df['h2'].values.astype(float)[:n_train]
    y_h2_test = df['h2'].values.astype(float)[n_train:]

    model=LinearRegression()
    model.fit(X_h2_train, y_h2_train)
    res_h2=model.predict(X_h2_test)

    predicted=plt.plot(range(n_samples-n_train), res_h2, label='predicted')
    real=plt.plot(range(n_samples-n_train), y_h2_test, label='real')
    plt.legend()
    plt.title('HORIZON2: difference')
    plt.show()

    real_level=df['level'].values.astype(float)[n_train:]
    predicted_level=[real_level[0]]
    for i in range(1, n_samples-n_train):
        predicted_level.append(res_h2[i]+predicted_level[-1])

    predicted_level_line=plt.plot(range(n_samples-n_train), predicted_level, label='predicted')
    real_level_line=plt.plot(range(n_samples-n_train), real_level, label='real')
    plt.legend()
    plt.title('HORIZON2: level')
    plt.show()

    #horizon 3
    X_h3_train = df_X[selected_features_h3].values.astype(float)[:n_train, :]
    X_h3_test = df_X[selected_features_h3].values.astype(float)[n_train:, :]
    y_h3_train = df['h3'].values.astype(float)[:n_train]
    y_h3_test = df['h3'].values.astype(float)[n_train:]

    model=LinearRegression()
    model.fit(X_h3_train, y_h3_train)
    res_h3=model.predict(X_h3_test)

    predicted=plt.plot(range(n_samples-n_train), res_h3, label='predicted')
    real=plt.plot(range(n_samples-n_train), y_h3_test, label='real')
    plt.legend()
    plt.title('HORIZON3: difference')
    plt.show()

    real_level=df['level'].values.astype(float)[n_train:]
    predicted_level=[real_level[0]]
    for i in range(1, n_samples-n_train):
        predicted_level.append(res_h3[i]+predicted_level[-1])

    predicted_level_line=plt.plot(range(n_samples-n_train), predicted_level, label='predicted')
    real_level_line=plt.plot(range(n_samples-n_train), real_level, label='real')
    plt.legend()
    plt.title('HORIZON3: level')
    plt.show()

    #horizon 5
    X_h5_train = df_X[selected_features_h5].values.astype(float)[:n_train, :]
    X_h5_test = df_X[selected_features_h5].values.astype(float)[n_train:, :]
    y_h5_train = df['h5'].values.astype(float)[:n_train]
    y_h5_test = df['h5'].values.astype(float)[n_train:]

    model=LinearRegression()
    model.fit(X_h5_train, y_h5_train)
    res_h5=model.predict(X_h5_test)

    predicted=plt.plot(range(n_samples-n_train), res_h5, label='predicted')
    real=plt.plot(range(n_samples-n_train), y_h5_test, label='real')
    plt.legend()
    plt.title('HORIZON5: difference')
    plt.show()

    real_level=df['level'].values.astype(float)[n_train:]
    predicted_level=[real_level[0]]
    for i in range(1, n_samples-n_train):
        predicted_level.append(res_h5[i]+predicted_level[-1])

    predicted_level_line=plt.plot(range(n_samples-n_train), predicted_level, label='predicted')
    real_level_line=plt.plot(range(n_samples-n_train), real_level, label='real')
    plt.legend()
    plt.title('HORIZON5: level')
    plt.show()

## Gradient boost

In [ ]:
n_folds=10

#crete target values for five different horizons
#select columns with features
X_h1 = df_X[selected_features].values.astype(float)
X_h2 = df_X[selected_features].values.astype(float)
X_h3 = df_X[selected_features].values.astype(float)
X_h5 = df_X[selected_features].values.astype(float)


def r2_calculation_gradient_boost():
    #splits data considering it is a time series
    timeSeriesCV=TimeSeriesSplit(n_splits=n_folds)
    
    model=GradientBoostingRegressor(n_estimators=100)
    cvs=cross_val_score(model, X_h1, y_h1, scoring='r2', cv=timeSeriesCV)
    print("h1:")
    print("R2: ", cvs.mean(), " stdev: ", cvs.std())

    model=GradientBoostingRegressor(n_estimators=100)
    cvs=cross_val_score(model, X_h2, y_h2, scoring='r2', cv=timeSeriesCV)
    print("h2:")
    print("R2: ", cvs.mean(), " stdev: ", cvs.std())

    model=GradientBoostingRegressor(n_estimators=100)
    cvs=cross_val_score(model, X_h3, y_h3, scoring='r2', cv=timeSeriesCV)
    print("h3:")
    print("R2: ", cvs.mean(), " stdev: ", cvs.std())

    model=GradientBoostingRegressor(n_estimators=100)
    cvs=cross_val_score(model, X_h5, y_h5, scoring='r2', cv=timeSeriesCV)
    print("h5:")
    print("R2: ", cvs.mean(), " stdev: ", cvs.std())
    
def grid_search_gradint_boost():
    param_search={'max_depth': [10, 50, 70, 100, None], 'max_features': ['auto', 'sqrt'], 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],}
    model=GradientBoostingRegressor(n_estimators=10)
    gsearch = GridSearchCV(estimator=model, cv=timeSeriesCV, param_grid=param_search, verbose=20, scoring='r2', n_jobs=7)
    gsearch.fit(X, y_h1)

    #print results
    print(gsearch.best_score_)
    print(gsearch.best_params_)

### displaying results

In [ ]:
def display_results_gradient_boost():    
    n_samples=df_X.shape[0]
    n_train=int(n_samples*0.9)

    #horizon 1
    X_h1_train = df_X[selected_features_h1].values.astype(float)[:n_train, :]
    X_h1_test = df_X[selected_features_h1].values.astype(float)[n_train:, :]
    y_h1_train = df['h1'].values.astype(float)[:n_train]
    y_h1_test = df['h1'].values.astype(float)[n_train:]

    model=GradientBoostingRegressor(n_estimators=100)
    model.fit(X_h1_train, y_h1_train)
    res_h1=model.predict(X_h1_test)

    predicted=plt.plot(range(n_samples-n_train), res_h1, label='predicted')
    real=plt.plot(range(n_samples-n_train), y_h1_test, label='real')
    plt.legend()
    plt.title('HORIZON1: difference')
    plt.show()

    real_level=df['level'].values.astype(float)[n_train:]
    predicted_level=[real_level[0]]
    for i in range(1, n_samples-n_train):
        predicted_level.append(res_h1[i]+predicted_level[-1])

    predicted_level_line=plt.plot(range(n_samples-n_train), predicted_level, label='predicted')
    real_level_line=plt.plot(range(n_samples-n_train), real_level, label='real')
    plt.legend()
    plt.title('HORIZON1: level')
    plt.show()

    #horizon 2
    X_h2_train = df_X[selected_features_h2].values.astype(float)[:n_train, :]
    X_h2_test = df_X[selected_features_h2].values.astype(float)[n_train:, :]
    y_h2_train = df['h2'].values.astype(float)[:n_train]
    y_h2_test = df['h2'].values.astype(float)[n_train:]

    model=GradientBoostingRegressor(n_estimators=100)
    model.fit(X_h2_train, y_h2_train)
    res_h2=model.predict(X_h2_test)

    predicted=plt.plot(range(n_samples-n_train), res_h2, label='predicted')
    real=plt.plot(range(n_samples-n_train), y_h2_test, label='real')
    plt.legend()
    plt.title('HORIZON2: difference')
    plt.show()

    real_level=df['level'].values.astype(float)[n_train:]
    predicted_level=[real_level[0]]
    for i in range(1, n_samples-n_train):
        predicted_level.append(res_h2[i]+predicted_level[-1])

    predicted_level_line=plt.plot(range(n_samples-n_train), predicted_level, label='predicted')
    real_level_line=plt.plot(range(n_samples-n_train), real_level, label='real')
    plt.legend()
    plt.title('HORIZON2: level')
    plt.show()

    #horizon 3
    X_h3_train = df_X[selected_features_h3].values.astype(float)[:n_train, :]
    X_h3_test = df_X[selected_features_h3].values.astype(float)[n_train:, :]
    y_h3_train = df['h3'].values.astype(float)[:n_train]
    y_h3_test = df['h3'].values.astype(float)[n_train:]

    model=GradientBoostingRegressor(n_estimators=100)
    model.fit(X_h3_train, y_h3_train)
    res_h3=model.predict(X_h3_test)

    predicted=plt.plot(range(n_samples-n_train), res_h3, label='predicted')
    real=plt.plot(range(n_samples-n_train), y_h3_test, label='real')
    plt.legend()
    plt.title('HORIZON3: difference')
    plt.show()

    real_level=df['level'].values.astype(float)[n_train:]
    predicted_level=[real_level[0]]
    for i in range(1, n_samples-n_train):
        predicted_level.append(res_h3[i]+predicted_level[-1])

    predicted_level_line=plt.plot(range(n_samples-n_train), predicted_level, label='predicted')
    real_level_line=plt.plot(range(n_samples-n_train), real_level, label='real')
    plt.legend()
    plt.title('HORIZON3: level')
    plt.show()

    #horizon 5
    X_h5_train = df_X[selected_features_h5].values.astype(float)[:n_train, :]
    X_h5_test = df_X[selected_features_h5].values.astype(float)[n_train:, :]
    y_h5_train = df['h5'].values.astype(float)[:n_train]
    y_h5_test = df['h5'].values.astype(float)[n_train:]

    model=GradientBoostingRegressor(n_estimators=100)
    model.fit(X_h5_train, y_h5_train)
    res_h5=model.predict(X_h5_test)

    predicted=plt.plot(range(n_samples-n_train), res_h5, label='predicted')
    real=plt.plot(range(n_samples-n_train), y_h5_test, label='real')
    plt.legend()
    plt.title('HORIZON5: difference')
    plt.show()

    real_level=df['level'].values.astype(float)[n_train:]
    predicted_level=[real_level[0]]
    for i in range(1, n_samples-n_train):
        predicted_level.append(res_h5[i]+predicted_level[-1])

    predicted_level_line=plt.plot(range(n_samples-n_train), predicted_level, label='predicted')
    real_level_line=plt.plot(range(n_samples-n_train), real_level, label='real')
    plt.legend()
    plt.title('HORIZON5: level')
    plt.show()

# Surface water level prediction

## The Data

Surface water level sensors with corresponding weather data from 2010-01-01 to 2017-12-31:
- `2250` // Otiški Vrh I (Meža)
- `2530` // Ruta (Radoljna)
- `2620` // Loče (Dravinja)
- `3200` // Sveti Janez (Sava Bohinjka)
- `3250` // Bodešče (Sava Bohinjka)
- `3400` // Mlino I (Jezernica)
- `4200` // Suha I (Sora)
- `4230` // Zminec (Poljanska Sora)
- `4270` // Železniki (Selška Sora)
- `4450` // Domžale (Mlinščica-Kanal)
- `4515` // Vir (Rača)
- `4520` // Podrečje (Rača)
- `4570` // Topole (Pšata)
- `4575` // Loka (Pšata)
- `4650` // Žebnik (Sopota)
- `4770` // Sodna vas II (Mestinjščica)
- `5040` // Kamin (Ljubljanica)
- `5078` // Moste I (Ljubljanica)
- `5330` // Borovnica (Borovniščica)
- `5425` // Iška vas (Iška)
- `5500` // Dvor (Gradaščica)
- `6060` // Nazarje (Savinja)
- `6068` // Letuš I (Savinja)
- `6200` // Laško I (Savinja)
- `6220` // Luče (Lučnica)
- `6300` // Šoštanj (Paka)
- `6340` // Rečica (Paka)
- `6350` // Škale (Lepena)
- `6385` // Pesje IV (Lepena)
- `6400` // Škale (Sopota)
- `8454` // Cerkno III (Cerknica)
- `8565` // Dolenje (Vipava)

Data columns:
- date
- level,day_time
- precipitation
- snow_accumulation
- temperature_avg
- temperature_min
- temperature_max
- cloud_cover_avg
- cloud_cover_min
- cloud_cover_max
- dew_point_avg
- dew_point_min
- dew_point_max
- humidity_avg
- humidity_min
- humidity_max
- pressure_avg
- pressure_min
- pressure_max
- precipitation_probability_avg
- precipitation_probability_min
- precipitation_probability_max
- precipitation_intensity_avg
- precipitation_intensity_min
- precipitation_intensity_ma

## Playground

In [ ]:
r2_calculation_random_forest()
#display_results_random_forest()

In [ ]:
r2_calculation_gradient_boost()
display_results_gradient_boost()

In [ ]:
r2_calculation_linear_regression()
display_results_linear_regression()

In [ ]:
grid_search_random_forest()

In [ ]:
display(selected_features_h1)

In [ ]:
n_folds=10
timeSeriesCV=TimeSeriesSplit(n_splits=n_folds)
    
    
for v in values:
    selected_features_h1 =feat.iloc[:, v.genes].columns
    X_h1 = df_X[selected_features_h1].values.astype(float)
    model=LinearRegression(normalize=True)
    #model=RandomForestRegressor(n_estimators=100)
    cvs=cross_val_score(model, X_h1, y_h1, scoring='r2', cv=timeSeriesCV)
    print("h1:")
    print("R2: ", cvs.mean(), " stdev: ", cvs.std())
    



In [ ]:
timeSeriesCV=TimeSeriesSplit(n_splits=n_folds)
object = pd.read_pickle(r'log/h1_tree/generation_1000.pickle')
le = preprocessing.LabelEncoder()
le.fit(df['h1'].values.astype(float))
#y_h1=le.transform(y_h1)
print(y_h1)
values=object.front.values()
for v in values:
    selected_features_h1 =feat.iloc[:, v.genes].columns
    X_h1 = df_X[selected_features_h1].values.astype(float)
    #model=LinearRegression(normalize=True)
    model=DecisionTreeClassifier()
    cvs=cross_val_score(model, X_h1, y_h1, scoring='r2', cv=timeSeriesCV)
    print("h1:")
    print("R2: ", cvs.mean(), " stdev: ", cvs.std())